In [1]:
from bs4 import BeautifulSoup
import os
from function import read_list_webs

https://hanoi.fpt.edu.vn/sinh-vien-dh-fpt-tich-hop-do-hoa-cong-nghe-va-truyen-thong-trong-san-pham-tot-nghiep.html
https://hanoi.fpt.edu.vn/dai-hoc-fpt-co-3-giang-vien-dat-giai-thuong-giang-vien-cao-cap-toan-cau.html
https://hanoi.fpt.edu.vn/nam-sinh-fpt-nam-xa-luyen-street-workout-moi-ngay-vi-truong-dau-tu-san-tap-xin-the-co-ma.html
https://hanoi.fpt.edu.vn/hanh-trinh-buoc-ra-the-gioi-cua-nu-sinh-dai-hoc-fpt.html
https://hanoi.fpt.edu.vn/nhin-lai-dem-nhac-ki-niem-10-nam-thanh-lap-clb-melody-fptu-lang-dong-va-cam-xuc.html
https://hanoi.fpt.edu.vn/khai-giang-chuong-trinh-ojt-tai-nhat-ban-cua-sinh-vien-chuyen-nganh-ky-su-cau-noi-dai-hoc-fpt.html
https://hanoi.fpt.edu.vn/do-an-cua-sinh-vien-nganh-tri-tue-nhan-tao-se-duoc-len-song-o-hoi-thao-intelligent-systems-conference-9-2023.html
https://hanoi.fpt.edu.vn/sinh-vien-dh-fpt-tich-hop-do-hoa-cong-nghe-va-truyen-thong-trong-san-pham-tot-nghiep.html
https://hanoi.fpt.edu.vn/dai-hoc-fpt-co-3-giang-vien-dat-giai-thuong-giang-vien-cao-cap-toan-c

In [2]:
list_webs = read_list_webs(os.path.join(".", "list_webs.txt"))

In [3]:
list_webs

['https://hanoi.fpt.edu.vn/360-do-hoa-lac']

In [ ]:
page = 1
